# 1. Load required packages

In [6]:
import recordlinkage
from recordlinkage.index import Block
import pandas as pd

# 2. Load database 1 and database 2

In [32]:
df_a = pd.read_csv('datasets\dataset4a.csv', index_col='rec_id',  dtype='object', na_values=" ") 
df_a.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [33]:
df_b = pd.read_csv('datasets\dataset4b.csv', index_col='rec_id', dtype='object', na_values=" ")
df_b.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


# 3. Blocking

In [34]:
df_a.columns = df_a.columns.str.strip()
df_b.columns = df_b.columns.str.strip()
indexer = recordlinkage.Index()
# indexer.full()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
pairs = indexer.index(df_a, df_b)
pairs

MultiIndex([(  'rec-0-org',    'rec-0-dup-0'),
            (  'rec-0-org', 'rec-1505-dup-0'),
            (  'rec-0-org', 'rec-1636-dup-0'),
            (  'rec-0-org', 'rec-2074-dup-0'),
            (  'rec-0-org', 'rec-2323-dup-0'),
            (  'rec-0-org', 'rec-2683-dup-0'),
            (  'rec-0-org', 'rec-2724-dup-0'),
            (  'rec-0-org', 'rec-2894-dup-0'),
            (  'rec-0-org', 'rec-3657-dup-0'),
            (  'rec-0-org', 'rec-4514-dup-0'),
            ...
            ('rec-999-org', 'rec-4649-dup-0'),
            ('rec-999-org', 'rec-4781-dup-0'),
            ('rec-999-org', 'rec-4941-dup-0'),
            ('rec-999-org', 'rec-4961-dup-0'),
            ('rec-999-org',   'rec-56-dup-0'),
            ('rec-999-org',  'rec-675-dup-0'),
            ('rec-999-org',  'rec-859-dup-0'),
            ('rec-999-org',   'rec-91-dup-0'),
            ('rec-999-org',  'rec-911-dup-0'),
            ('rec-999-org',  'rec-999-dup-0')],
           names=['rec_id_1', 'rec_id_2'], 

# 4. Rule-based algorithm 

In [35]:
compare_cl = recordlinkage.Compare()
compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(pairs, df_a, df_b)

In [36]:
features

given_name  surname  date_of_birth  suburb  state  \
rec_id_1    rec_id_2                                                            
rec-0-org   rec-0-dup-0              1      1.0              1       1      1   
            rec-1505-dup-0           0      1.0              0       0      0   
            rec-1636-dup-0           0      1.0              0       0      0   
            rec-2074-dup-0           0      1.0              0       0      1   
            rec-2323-dup-0           1      0.0              0       0      0   
...                                ...      ...            ...     ...    ...   
rec-999-org rec-675-dup-0            1      0.0              0       0      1   
            rec-859-dup-0            0      1.0              0       0      0   
            rec-91-dup-0             1      0.0              0       0      0   
            rec-911-dup-0            0      1.0              0       0      1   
            rec-999-dup-0            1      1.0              1       1      1   

                            address_1  
rec_id_1    rec_id_2                   
rec-0-org   rec-0-dup-0           1.0  
            rec-1505-dup-0        0.0  
            rec-1636-dup-0        0.0  
            rec-2074-dup-0        0.0  
            rec-2323-dup-0        0.0  
...                               ...  
rec-999-org rec-675-dup-0         0.0  
            rec-859-dup-0         0.0  
            rec-91-dup-0          0.0  
            rec-911-dup-0         0.0  
            rec-999-dup-0         1.0  

[160159 rows x 6 columns]

In [42]:
features[features.sum(axis=1) > 3]

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-0-org,rec-0-dup-0,1,1.0,1,1,1,1.0
rec-1-org,rec-1-dup-0,1,1.0,1,1,1,1.0
rec-10-org,rec-10-dup-0,0,1.0,1,1,1,1.0
rec-100-org,rec-100-dup-0,1,1.0,0,1,1,1.0
rec-1000-org,rec-1000-dup-0,1,1.0,1,1,1,1.0
...,...,...,...,...,...,...,...
rec-995-org,rec-995-dup-0,1,1.0,1,1,1,1.0
rec-996-org,rec-996-dup-0,1,1.0,1,1,1,1.0
rec-997-org,rec-997-dup-0,1,1.0,1,1,1,1.0


# 5. Supervised algorithm

In [38]:
intercept = -11.0
coefficients = [1.5, 1.5, 8.0, 6.0, 2.5, 6.5]

In [39]:
logreg = recordlinkage.LogisticRegressionClassifier(
    coefficients=coefficients, intercept=intercept)
links = logreg.predict(features)

In [40]:
links

MultiIndex([(   'rec-0-org',    'rec-0-dup-0'),
            (   'rec-1-org',    'rec-1-dup-0'),
            (  'rec-10-org',   'rec-10-dup-0'),
            ( 'rec-100-org',  'rec-100-dup-0'),
            ('rec-1000-org', 'rec-1000-dup-0'),
            ('rec-1001-org', 'rec-1001-dup-0'),
            ('rec-1002-org', 'rec-1002-dup-0'),
            ('rec-1003-org', 'rec-1003-dup-0'),
            ('rec-1004-org', 'rec-1004-dup-0'),
            ('rec-1005-org', 'rec-1005-dup-0'),
            ...
            (  'rec-99-org',   'rec-99-dup-0'),
            ( 'rec-990-org',  'rec-990-dup-0'),
            ( 'rec-991-org',  'rec-991-dup-0'),
            ( 'rec-993-org',  'rec-993-dup-0'),
            ( 'rec-994-org',  'rec-994-dup-0'),
            ( 'rec-995-org',  'rec-995-dup-0'),
            ( 'rec-996-org',  'rec-996-dup-0'),
            ( 'rec-997-org',  'rec-997-dup-0'),
            ( 'rec-998-org',  'rec-998-dup-0'),
            ( 'rec-999-org',  'rec-999-dup-0')],
           names=['rec_